In [0]:
!pip install -q sklearn

In [0]:
%tensorflow_version 2.x

##Necessary Dependicies

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

#Reading dataset

In [0]:
dataset = pd.read_csv('/content/Iris.csv')
shuffle_df = dataset.sample(frac=1)

# Define a size for your train set
train_size = int(0.7 * len(dataset))

# Split your dataset
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

train_set = train_set.drop(['Id'],axis=1)
test_set = test_set.drop(['Id'],axis=1)

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_set['Species'] = le.fit_transform(train_set['Species'])
test_set['Species'] = le.fit_transform(test_set['Species'])

In [22]:
train_set

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
39,5.1,3.4,1.5,0.2,0
97,6.2,2.9,4.3,1.3,1
90,5.5,2.6,4.4,1.2,1
50,7.0,3.2,4.7,1.4,1
138,6.0,3.0,4.8,1.8,2
...,...,...,...,...,...
105,7.6,3.0,6.6,2.1,2
72,6.3,2.5,4.9,1.5,1
10,5.4,3.7,1.5,0.2,0
45,4.8,3.0,1.4,0.3,0


In [23]:
train_y = train_set.pop('Species')
test_y = test_set.pop('Species')
train_set.head()


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
39,5.1,3.4,1.5,0.2
97,6.2,2.9,4.3,1.3
90,5.5,2.6,4.4,1.2
50,7.0,3.2,4.7,1.4
138,6.0,3.0,4.8,1.8


In [24]:
len(set(train_y))

3

In [25]:
train_set.shape

(105, 4)

In [26]:
test_set.shape

(45, 4)

In [0]:
def input_fn(features,labels,training = True,batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [28]:
my_feature_column = []
for key in train_set.keys():
  my_feature_column.append(tf.feature_column.numeric_column(key=key))
print(my_feature_column)

[NumericColumn(key='SepalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [29]:
mkdir outputs_dir

mkdir: cannot create directory ‘outputs_dir’: File exists


## Building the model

In [30]:
calssifier = tf.estimator.DNNClassifier(feature_columns=my_feature_column,hidden_units=[30,10],n_classes=3,model_dir='/content/outputs_dir')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/outputs_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
calssifier.train(input_fn = lambda: input_fn(train_set,train_y,training=True),steps = 5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/outputs_dir/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into /content/outputs_dir/model.ckpt.
INFO:tensorflow:Calling check

In [35]:
eval_results = calssifier.evaluate(input_fn=lambda: input_fn(test_set,test_y,training = False))
print('\n Test set accuracy :{accuracy:0.3f}\n '.format(**eval_results))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-31T12:55:58Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/outputs_dir/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26631s
INFO:tensorflow:Finished evaluation at 2020-03-31-12:55:59
INFO:tensorflow:Saving dict for global step 15000: accuracy = 1.0, average

In [0]:
Species = ['setosa','versicolor','virginica']

In [48]:
def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
predict = {}

print("please type numeric values as prompted")

for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit():valid = False

    predict[feature] = [float(val)]

predictions = calssifier.predict(input_fn =lambda: input_fn(predict))  
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print(Species[class_id])
  print(100 * probability)
  # print('prediction is "{}"({:.1f%})'.format(Species[class_id],100 * probability))

please type numeric values as prompted
SepalLengthCm: 2.4
SepalWidthCm: 2.6
PetalLengthCm: 6.3
PetalWidthCm: 6.4
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/outputs_dir/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
virginica
95.93827724456787
